# NYC Shooting Incidents Analysis

In [1]:
# import packages
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import matplotlib.pylab as plt
import math
import os 
import seaborn as sns #for styling the plots
import warnings
import urllib.request
from scipy import stats
import statsmodels.formula.api as smf
%matplotlib inline

In [22]:
Data = pd.read_csv('light_bk.csv')

In [8]:
pd.set_option('display.max_columns', None)

In [23]:
Data.head()

,Unnamed: 0,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,Street Name,Cross Street 1,Cross Street 2,Intersection Street 1,Intersection Street 2,Address Type,City,Landmark,Facility Type,Status,Due Date,Resolution Description,Resolution Action Updated Date,Community Board,BBL,Borough,X Coordinate (State Plane),Y Coordinate (State Plane),Open Data Channel Type,Park Facility Name,Park Borough,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location
0,52,33769340,07/06/2016 01:35:00 AM,07/21/2016 12:00:00 AM,DOT,Department of Transportation,Street Light Condition,Traffic Signal Light,NaN,NaN,NaN,NaN,ADELPHI ST,PARK AVE,ADELPHI ST,PARK AVE,INTERSECTION,NaN,NaN,NaN,Closed,NaN,Service Request status for this request is ava...,07/21/2016 12:00:00 AM,Unspecified BROOKLYN,NaN,BROOKLYN,NaN,NaN,UNKNOWN,Unspecified,BROOKLYN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,77,33880765,07/19/2016 12:11:00 PM,07/19/2016 12:11:00 PM,DOT,Department of Transportation,Street Light Condition,Street Light Out,NaN,11203.0,NaN,NaN,UTICA AVE,FOSTER AVE,UTICA AVENUE,FOSTER AVENUE,INTERSECTION,BROOKLYN,NaN,NaN,Closed,NaN,Service Request status for this request is ava...,07/18/2016 12:11:00 PM,17 BROOKLYN,NaN,BROOKLYN,1003925.0,172379.0,UNKNOWN,Unspecified,BROOKLYN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.639795,-73.929107,"(40.63979528864854, -73.92910706900996)"
2,102,33886470,07/20/2016 11:58:00 AM,07/20/2016 11:58:00 AM,DOT,Department of Transportation,Street Light Condition,Street Light Out,NaN,NaN,NaN,NaN,95 ST E,AVE K,95 ST E,AVE K,INTERSECTION,NaN,NaN,NaN,Closed,NaN,Service Request status for this request is ava...,07/19/2016 11:58:00 AM,Unspecified BROOKLYN,NaN,BROOKLYN,NaN,NaN,UNKNOWN,Unspecified,BROOKLYN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,108,33763371,07/05/2016 05:50:00 PM,07/06/2016 12:08:00 PM,DOT,Department of Transportation,Street Light Condition,Fixture/Luminaire Out Of Position,NaN,11210.0,3402 AVENUE L,AVENUE L,34 ST E,35 ST E,NaN,NaN,ADDRESS,BROOKLYN,NaN,NaN,Closed,NaN,Service Request status for this request is ava...,07/06/2016 12:08:00 PM,18 BROOKLYN,3.076520e+09,BROOKLYN,1000342.0,166046.0,UNKNOWN,Unspecified,BROOKLYN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.622420,-73.942032,"(40.6224197807114, -73.94203242567343)"
4,133,33908989,07/22/2016 11:02:00 AM,08/31/2016 12:14:00 PM,DOT,Department of Transportation,Street Light Condition,Street Light Out,NaN,11215.0,NaN,NaN,CARROLL ST,5 AVE,CARROLL STREET,5 AVENUE,INTERSECTION,BROOKLYN,NaN,NaN,Closed,NaN,Service Request status for this request is ava...,08/31/2016 12:14:00 PM,06 BROOKLYN,NaN,BROOKLYN,989430.0,185251.0,UNKNOWN,Unspecified,BROOKLYN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.675146,-73.981326,"(40.67514644049858, -73.98132554022042)"


In [34]:
light_location = Data.drop_duplicates(subset=['Location'])

In [36]:
light_location.dropna(subset=['Location'])

,Unnamed: 0,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,Street Name,Cross Street 1,Cross Street 2,Intersection Street 1,Intersection Street 2,Address Type,City,Landmark,Facility Type,Status,Due Date,Resolution Description,Resolution Action Updated Date,Community Board,BBL,Borough,X Coordinate (State Plane),Y Coordinate (State Plane),Open Data Channel Type,Park Facility Name,Park Borough,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location
1,77,33880765,07/19/2016 12:11:00 PM,07/19/2016 12:11:00 PM,DOT,Department of Transportation,Street Light Condition,Street Light Out,NaN,11203.0,NaN,NaN,UTICA AVE,FOSTER AVE,UTICA AVENUE,FOSTER AVENUE,INTERSECTION,BROOKLYN,NaN,NaN,Closed,NaN,Service Request status for this request is ava...,07/18/2016 12:11:00 PM,17 BROOKLYN,NaN,BROOKLYN,1003925.0,172379.0,UNKNOWN,Unspecified,BROOKLYN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.639795,-73.929107,"(40.63979528864854, -73.92910706900996)"
3,108,33763371,07/05/2016 05:50:00 PM,07/06/2016 12:08:00 PM,DOT,Department of Transportation,Street Light Condition,Fixture/Luminaire Out Of Position,NaN,11210.0,3402 AVENUE L,AVENUE L,34 ST E,35 ST E,NaN,NaN,ADDRESS,BROOKLYN,NaN,NaN,Closed,NaN,Service Request status for this request is ava...,07/06/2016 12:08:00 PM,18 BROOKLYN,3.076520e+09,BROOKLYN,1000342.0,166046.0,UNKNOWN,Unspecified,BROOKLYN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.622420,-73.942032,"(40.6224197807114, -73.94203242567343)"
4,133,33908989,07/22/2016 11:02:00 AM,08/31/2016 12:14:00 PM,DOT,Department of Transportation,Street Light Condition,Street Light Out,NaN,11215.0,NaN,NaN,CARROLL ST,5 AVE,CARROLL STREET,5 AVENUE,INTERSECTION,BROOKLYN,NaN,NaN,Closed,NaN,Service Request status for this request is ava...,08/31/2016 12:14:00 PM,06 BROOKLYN,NaN,BROOKLYN,989430.0,185251.0,UNKNOWN,Unspecified,BROOKLYN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.675146,-73.981326,"(40.67514644049858, -73.98132554022042)"
6,408,33866914,07/18/2016 12:08:00 PM,07/15/2016 12:08:00 PM,DOT,Department of Transportation,Street Light Condition,Street Light Out,NaN,11238.0,NaN,NaN,LEXINGTON AVE,CLASSON AVE,LEXINGTON AVENUE,CLASSON AVENUE,INTERSECTION,BROOKLYN,NaN,NaN,Pending,NaN,Service Request status for this request is ava...,07/15/2016 12:08:00 PM,02 BROOKLYN,NaN,BROOKLYN,995463.0,189456.0,UNKNOWN,Unspecified,BROOKLYN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.686683,-73.959569,"(40.686682623345476, -73.95956892856266)"
7,602,33812636,07/11/2016 12:08:00 PM,07/08/2016 12:08:00 PM,DOT,Department of Transportation,Street Light Condition,Street Light Out,NaN,11214.0,NaN,NaN,CROPSEY AVE,BAY 13 ST,CROPSEY AVENUE,BAY 13 STREET,INTERSECTION,BROOKLYN,NaN,NaN,Pending,NaN,Service Request status for this request is ava...,07/08/2016 12:08:00 PM,11 BROOKLYN,NaN,BROOKLYN,981057.0,159607.0,UNKNOWN,Unspecified,BROOKLYN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.604760,-74.011499,"(40.60476009377737, -74.01149897423922)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297583,35741420,33699504,06/27/2016 09:48:00 PM,06/28/2016 11:26:00 AM,DOT,Department of Transportation,Street Light Condition,Street Light Lamp Missing,NaN,11233.0,24 SUMPTER STREET,SUMPTER STREET,FULTON ST,PATCHEN AVE,NaN,NaN,ADDRESS,BROOKLYN,NaN,NaN,Closed,NaN,Service Request status for this request is ava...,06/28/2016 11:26:00 AM,03 BROOKLYN,NaN,BROOKLYN,1004714.0,186764.0,UNKNOWN,Unspecified,BROOKLYN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.679277,-73.926220,"(40.67927714486069, -73.92622049075123)"
297586,35741856,33660519,06/22/2016 01:03:00 PM,06/22/2016 01:54:00 PM,DOT,Department of Transportation,Street Light Condition,Street Light Out,NaN,11203.0,3913 SNYDER AVENUE,SNYDER AVENUE,39 ST E,40 ST E,SNYDER AVE,39 ST E,INTERSECTION,BROOKLYN,NaN,NaN,Closed,NaN,Service Reque

In [37]:
light_location.columns

Index(['Unnamed: 0', 'Unique Key', 'Created Date', 'Closed Date', 'Agency',
       'Agency Name', 'Complaint Type', 'Descriptor', 'Location Type',
       'Incident Zip', 'Incident Address', 'Street Name', 'Cross Street 1',
       'Cross Street 2', 'Intersection Street 1', 'Intersection Street 2',
       'Address Type', 'City', 'Landmark', 'Facility Type', 'Status',
       'Due Date', 'Resolution Description', 'Resolution Action Updated Date',
       'Community Board', 'BBL', 'Borough', 'X Coordinate (State Plane)',
       'Y Coordinate (State Plane)', 'Open Data Channel Type',
       'Park Facility Name', 'Park Borough', 'Vehicle Type',
       'Taxi Company Borough', 'Taxi Pick Up Location', 'Bridge Highway Name',
       'Bridge Highway Direction', 'Road Ramp', 'Bridge Highway Segment',
       'Latitude', 'Longitude', 'Location'],
      dtype='object')

In [42]:

lights_bk = light_location[['Borough','Incident Zip','Latitude','Longitude','Location']]

In [44]:
lights_bk.dropna(subset=['Location'])

,Borough,Incident Zip,Latitude,Longitude,Location
1,BROOKLYN,11203.0,40.639795,-73.929107,"(40.63979528864854, -73.92910706900996)"
3,BROOKLYN,11210.0,40.622420,-73.942032,"(40.6224197807114, -73.94203242567343)"
4,BROOKLYN,11215.0,40.675146,-73.981326,"(40.67514644049858, -73.98132554022042)"
6,BROOKLYN,11238.0,40.686683,-73.959569,"(40.686682623345476, -73.95956892856266)"
7,BROOKLYN,11214.0,40.604760,-74.011499,"(40.60476009377737, -74.01149897423922)"
...,...,...,...,...,...
297583,BROOKLYN,11233.0,40.679277,-73.926220,"(40.67927714486069, -73.92622049075123)"
297586,BROOKLYN,11203.0,40.649474,-73.940546,"(40.649474394659876, -73.94054580868084)"
297631,BROOKLYN,11221.0,40.687348,-73.915982,"(40.687347992177486, -73.91598199347952)"
297638,BROOKLYN,11237.0,40.699665,-73.913262,"(40.69966450134263, -73.91326164500046)"
